<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/EMA_RSI_Kompozit_Tarama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ GEREKLİ KURULUMLAR (Colab için)
# Genellikle bu ikisi yeterli olacaktır.
!pip install tradingview_screener==2.5.0 # Eğer kurulu değilse
!pip install git+https://github.com/rongardF/tvdatafeed # Eğer kurulu değilse
!pip install pandas ta numpy openpyxl tqdm tabulate # Ek kütüphaneler eğer kurulu değilse

# ✅ GEREKLİ KÜTÜPHANELERİ KONTROL ET & YÜKLE (Artık özel fonksiyonlara gerek yok)
# Import etmeye çalışacağız, hata verirse kurulum mesajı gösteririz.

# ✅ KÜTÜPHANELER
import pandas as pd
import numpy as np
import warnings
import ta
from tqdm import tqdm
from datetime import datetime
from tradingview_screener import get_all_symbols
from tvDatafeed import TvDatafeed, Interval

warnings.filterwarnings("ignore")
tv = TvDatafeed()


# ✅ EMA ve Koşul Tipi Seçimi
print("\n📌 EMA Tarama Ayarları:")
while True:
    try:
        ema1 = int(input("EMA1 Periyodu Giriniz (örn: 13): "))
        ema2 = int(input("EMA2 Periyodu Giriniz (örn: 34): "))
        ema3_input = input("EMA3 (isteğe bağlı): ").strip()
        ema3 = int(ema3_input) if ema3_input else None
        break
    except ValueError:
        print("❗ Geçerli sayı girin.")

# ✅ Tarama Zaman Periyodu
print("\n📆 Tarama Zaman Periyodunu Seçiniz:")
print("1 - Günlük\n2 - Haftalık\n3 - Aylık")
secim = input("Seçim (1/2/3): ")

if secim == "1":
    interval, interval_label = Interval.in_daily, "Günlük"
elif secim == "2":
    interval, interval_label = Interval.in_weekly, "Haftalık"
elif secim == "3":
    interval, interval_label = Interval.in_monthly, "Aylık"
else:
    interval, interval_label = Interval.in_daily, "Günlük"

# ✅ Tarama Modu
print("🎯 Tarama Koşulunu Seçin:")
print("1 - EMA1 EMA2'yi Yukarı Kesiyor")
print("2 - Fiyat EMA1'i Yukarı Kesiyor")
print("3 - Fiyat EMA1'e Yüzdesel Yakınlık (0-5%)")
tarama_kosulu = input("Seçiminiz (1/2/3): ").strip()


# ✅ Tarama Modu
print("\n📌 Kompozit Tarama Modunu Seçin:")
print("1 - 🔁 Tüm Kompozit Hisseleri Tara")
print("2 - 🔍 Belirli Kompozit Hisse(ler)i Tara (Bilgilendirme)")

mod_secim = input("Seçiminiz (1/2): ").strip()

if mod_secim == "2":
    hisse_giris = input("🎯 Hisse(ler)i girin (virgülle ayırın, örn: ASELS,KCHOL,THYAO): ").upper().replace(" ", "")
    Hisseler = hisse_giris.split(",")
else:
    Hisseler = sorted([s.replace('BIST:', '') for s in get_all_symbols('turkey')])

ema_bilgi = f"EMA1: {ema1}, EMA2: {ema2}" + (f", EMA3: {ema3}" if ema3 else "")
print(f"\n📈 Seçilen EMA'lar → {ema_bilgi} | Periyot: {interval_label}")

print(f"🔢 Toplam taranacak hisse sayısı: {len(Hisseler)}")

# ✅ TEKNİK ANALİZ FONKSİYONU (HACİM MODÜLLERİ EKLENDİ)
def get_teknik_analiz(df, ema1=13, ema2=34, ema3=None):
    yorumlar = []

    # RSI Analizi
    rsi = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    rsi_val = rsi.iloc[-1]
    rsi_yon = rsi.iloc[-1] - rsi.iloc[-2]
    if rsi_val > 50 and rsi_yon > 0:
        yorumlar.append("✅ RSI > 50 🔼")
    elif rsi_val < 50 and rsi_yon < 0:
        yorumlar.append("⚠️ RSI < 50 🔽")

    # EMA Analizi
    ema_1 = df['close'].ewm(span=ema1, adjust=False).mean()
    ema_2 = df['close'].ewm(span=ema2, adjust=False).mean()

    if ema_1.iloc[-2] < ema_2.iloc[-2] and ema_1.iloc[-1] > ema_2.iloc[-1]:
        yorumlar.append(f"✅ EMA{ema1}, EMA{ema2}'yi yukarı kesti")
    elif ema_1.iloc[-1] < ema_2.iloc[-1]:
        yorumlar.append(f"🔻 EMA{ema1} EMA{ema2}'nin altında")
    else:
        yorumlar.append(f"⏸ EMA{ema1}, EMA{ema2}'nin üstünde (kesişim yok)")

    # EMA3 Kontrolü (Varsa)
    if ema3 is not None:
        ema_3 = df['close'].ewm(span=ema3, adjust=False).mean()
        if df['close'].iloc[-1] > ema_3.iloc[-1]:
            yorumlar.append(f"📈 Fiyat EMA{ema3}'ün üstünde")
        else:
            yorumlar.append(f"📉 Fiyat EMA{ema3}'ün altında")


    # 📉 MACD Analizi (Yön, Bölge ve Momentum Kombinasyonu)
    macd_line = ta.trend.macd(df['close'], window_slow=26, window_fast=12)
    macd_signal = ta.trend.macd_signal(df['close'], window_slow=26, window_fast=12)
    macd_diff = ta.trend.macd_diff(df['close'], window_slow=26, window_fast=12)

    if macd_diff.iloc[-1] > 0 and macd_diff.iloc[-2] < 0:
        yorumlar.append("✅ MACD Al (Yukarı Kesişim)")

    if macd_line.iloc[-1] > 0:
        yorumlar.append("📈 MACD Pozitif Bölgede (0 Üzeri)")
    elif macd_diff.iloc[-1] > 0:
        yorumlar.append("🔄 MACD Sinyalin Üstünde (Ama 0 Altı)")

    if macd_line.iloc[-1] > macd_line.iloc[-2] and macd_signal.iloc[-1] > macd_signal.iloc[-2]:
        yorumlar.append("💪 MACD Momentum Yukarı")
    elif macd_line.iloc[-1] < macd_line.iloc[-2] and macd_signal.iloc[-1] < macd_signal.iloc[-2]:
        yorumlar.append("⚠️ MACD Momentum Zayıflıyor")


    # HACİM Analizi
    son_hacim = df['volume'].iloc[-1]
    hacim_ort_10 = df['volume'].rolling(10).mean().iloc[-1]
    hacim_ort_20 = df['volume'].rolling(20).mean().iloc[-1]

    if son_hacim > 2 * hacim_ort_20:
        yorumlar.append(f"🧨 Hacim Patlaması (x{son_hacim/hacim_ort_20:.1f})")
    elif son_hacim > 1.5 * hacim_ort_10:
        yorumlar.append(f"📈 Hacim Artışı (x{son_hacim/hacim_ort_10:.1f})")

    # Hacim Yönü Analizi (Alış mı Satış mı?)
    if son_hacim > hacim_ort_20:
        if df['close'].iloc[-1] > df['open'].iloc[-1]:
            yorumlar.append("💚 Yeşil Hacim Artışı (Alış Baskın)")
        elif df['close'].iloc[-1] < df['open'].iloc[-1]:
            yorumlar.append("❤️ Kırmızı Hacim Artışı (Satış Baskın)")

    # Ek olarak: Yükseliş destekli hacim
    if (son_hacim > hacim_ort_20) and (df['close'].iloc[-1] > df['open'].iloc[-1]):
        yorumlar.append("💰 Yükseliş Destekli Hacim")


    # DMI (DI+ ve DI-) Kesişim Analizi
    adx = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=14)

    plus_di = adx.adx_pos()
    minus_di = adx.adx_neg()

    if plus_di.iloc[-2] < minus_di.iloc[-2] and plus_di.iloc[-1] > minus_di.iloc[-1]:
        yorumlar.append("✅ DI+ DI- Yukarı Kesti (Alış Güçleniyor)")
    elif plus_di.iloc[-2] > minus_di.iloc[-2] and plus_di.iloc[-1] < minus_di.iloc[-1]:
        yorumlar.append("⚠️ DI+ DI- Aşağı Kesti (Satış Güçleniyor)")


    # OBV Göstergesi
    obv = ta.volume.OnBalanceVolumeIndicator(close=df['close'], volume=df['volume']).on_balance_volume()
    if obv.iloc[-1] > obv.rolling(5).mean().iloc[-1]:
        yorumlar.append("📊 OBV Yükselişte")

    # Momentum (MOM) Analizi
    momentum = ta.momentum.ROCIndicator(df['close'], window=10).roc()
    momentum_val = momentum.iloc[-1]

    if momentum_val > 0:
        yorumlar.append("🚀 Momentum Pozitif")
    else:
        yorumlar.append("⚡ Momentum Negatif")


    # 📈 MOMENTUM (ROC) Analizi
    try:
        momentum = ta.momentum.ROCIndicator(df['close'], window=10).roc()
        momentum_val = momentum.iloc[-1]

        if momentum_val > 5:
            yorumlar.append("🚀 Momentum Çok Güçlü (>+5%)")
        elif 0 < momentum_val <= 5:
            yorumlar.append("🔵 Momentum Pozitif (Alıcılı)")
        elif -5 <= momentum_val < 0:
            yorumlar.append("🟠 Momentum Zayıf (Satıcılı)")
        else:  # momentum_val < -5
            yorumlar.append("🔻 Momentum Çok Zayıf (<-5%)")
    except Exception as e:
        yorumlar.append("⚠️ Momentum Hesaplanamadı")


    # 📊 DMI (ADX) Analizi
    try:
        adx = ta.trend.ADXIndicator(high=df['high'], low=df['low'], close=df['close'], window=14)
        adx_val = adx.adx().iloc[-1]
        plus_di = adx.adx_pos().iloc[-1]
        minus_di = adx.adx_neg().iloc[-1]

        if adx_val > 25:
            if plus_di > minus_di:
                yorumlar.append("📈 DMI: Güçlü Alış Trendi")
            else:
                yorumlar.append("📉 DMI: Güçlü Satış Trendi")
        else:
            yorumlar.append("⚡ DMI: Zayıf Trend (Kararsız)")
    except Exception as e:
        yorumlar.append("⚠️ DMI Hesaplanamadı")


    return " | ".join(yorumlar) if yorumlar else "❌ Belirgin Sinyal Yok"

# ✅ KOMPOZİT ANALİZ FONKSİYONU (FİYAT EMA'yı yukarı kesiyor mu?)
def analiz_yap_kompozit(hisse, interval, ema1, ema2, ema3=None, tarama_kosulu="1"):
    try:
        # 1️⃣ Verileri çek
        df_hisse = tv.get_hist(hisse, 'BIST', interval=interval, n_bars=200)
        df_xu100 = tv.get_hist("XU100", 'BIST', interval=interval, n_bars=200)
        if df_hisse is None or df_xu100 is None or df_hisse.empty or df_xu100.empty:
            return None

        # 2️⃣ Kompozit fiyat serisi
        kompozit = (df_hisse['close'] / df_xu100['close']) * 100
        fiyat = kompozit.iloc[-1]
        onceki_fiyat = kompozit.iloc[-2]

        # 3️⃣ EMA hesapla
        ema1_seri = kompozit.ewm(span=ema1, adjust=False).mean()
        ema2_seri = kompozit.ewm(span=ema2, adjust=False).mean()
        ema3_seri = kompozit.ewm(span=ema3, adjust=False).mean() if ema3 else None

        # 4️⃣ Tarama Koşulu Seçimi
        if tarama_kosulu == "1":  # EMA1 EMA2'yi yukarı kesiyor mu?
            if ema1_seri.iloc[-2] < ema2_seri.iloc[-2] and ema1_seri.iloc[-1] > ema2_seri.iloc[-1]:
                kriter = f"✅ EMA{ema1}, EMA{ema2}'yi yukarı kesti"
            else:
                return None

        elif tarama_kosulu == "2":  # Fiyat EMA1'i yukarı kesiyor mu?
            if onceki_fiyat < ema1_seri.iloc[-2] and fiyat > ema1_seri.iloc[-1]:
                kriter = f"✅ Fiyat EMA{ema1}'i yukarı kesti"
            else:
                return None

        elif tarama_kosulu == "3":  # Fiyat EMA1'e % yakın mı? (0-5%)
            uzaklik = abs(fiyat - ema1_seri.iloc[-1]) / ema1_seri.iloc[-1]
            if uzaklik <= 0.05:
                if fiyat > ema1_seri.iloc[-1]:
                    kriter = f"📈 Fiyat EMA{ema1}'in %5 ÜSTÜNDE (yakın)"
                else:
                    kriter = f"📉 Fiyat EMA{ema1}'in %5 ALTINDA (yakın)"
            else:
                return None

        else:
            return None


        # 5️⃣ Teknik sinyalleri al
        teknik_sinyaller = get_teknik_analiz(df_hisse, ema1, ema2, ema3)
        rsi = ta.momentum.RSIIndicator(close=df_hisse['close'], window=14).rsi()  # <<< BU SATIRI DA EKLE

        # 6️⃣ Sonuç döndür
        return {
            "Hisse": hisse,
            "Periyot": interval_label,
            "Fiyat": round(df_hisse['close'].iloc[-1], 2),
            f"EMA{ema2}": round(ema2_seri.iloc[-1], 2),
            "Kriter": kriter,
            "Teknik Sinyaller": teknik_sinyaller,
            "RSI": round(rsi.iloc[-1], 2)   # <<< BU SATIR YENİ EKLENDİ
        }

    except Exception as e:
        print(f"⚠️ {hisse} hata: {e}")
        return None


# 🎯 TARAMA BAŞLIYOR
sonuclar_kompozit = []  # 🔍 Tarama sonuçları burada birikecek

print("\n🔍 Kompozit Tarama Başladı...")

for hisse in tqdm(Hisseler, desc="⏳ Taranıyor"):
    sonuc = analiz_yap_kompozit(hisse, interval, ema1, ema2, ema3, tarama_kosulu)
    if sonuc:
        sonuclar_kompozit.append(sonuc)


# 📊 SONUÇLARI DÜZENLE
df_sonuc = pd.DataFrame(sonuclar_kompozit)

# ✅ PANDAS GÖRÜNTÜLEME AYARLARI
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1200)
pd.set_option('display.max_colwidth', 25)
pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.colheader_justify', 'left')

# 🎨 STİL FONKSİYONU
def format_tarama_sonucu(df):
    renk_kolon = [k for k in df.columns if k.startswith("EMA")]
    return (df.style
        .set_properties(**{'text-align': 'left', 'font-size': '12px'})
        .set_table_styles([{
            'selector': 'th',
            'props': [('background-color', '#2A5CAA'), ('color', 'white')]
        }])
        .background_gradient(subset=renk_kolon + ['Fiyat'], cmap='Blues')
        .format({col: '{:,.2f}₺' for col in renk_kolon + ['Fiyat']}))

# 📋 SONUÇ GÖSTER
if not df_sonuc.empty:
    df_sonuc = df_sonuc.sort_values(by='Fiyat', ascending=False)
    print(f"\n🎯 {len(df_sonuc)} hisse bulundu.")
    display(format_tarama_sonucu(df_sonuc.reset_index(drop=True)))

    # 🏆 EN YÜKSEK 10 FİYAT
    print("\n🏆 Fiyat Bazlı En Yüksek 10 Hisse")
    top_10 = df_sonuc.nlargest(10, 'Fiyat')
    top_10 = top_10.assign(Sıra=range(1, len(top_10) + 1)).set_index("Sıra")
    display(format_tarama_sonucu(top_10))

# ✅ RSI > 50 Üzeri Olan Hisseler
rsi_50_ustu = df_sonuc[df_sonuc["RSI"] > 50]
if not rsi_50_ustu.empty:
    print("\n✅ RSI 50 Üzeri Olan Hisseler")
    display(format_tarama_sonucu(rsi_50_ustu.reset_index(drop=True)))
else:
    print("\n⚠️ RSI 50 Üzeri hisse bulunamadı.")
    print("⚠️ Uygun hisse bulunamadı.")

    # 💾 Excel'e Kaydet
    now = datetime.now().strftime('%Y%m%d_%H%M')
    dosya_adi = f"kompozit_tarama_{interval_label}_{now}.xlsx"
    with pd.ExcelWriter(dosya_adi, engine='openpyxl') as writer:
        df_sonuc.to_excel(writer, sheet_name='Sonuclar', index=False)
        top_10.to_excel(writer, sheet_name='Top10', index=False)
    print(f"\n💾 Excel'e kaydedildi: {dosya_adi} ✅")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-8rt1pcpc
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-8rt1pcpc
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done



📌 EMA Tarama Ayarları:
EMA1 Periyodu Giriniz (örn: 13): 13
EMA2 Periyodu Giriniz (örn: 34): 34
EMA3 (isteğe bağlı): 55

📆 Tarama Zaman Periyodunu Seçiniz:
1 - Günlük
2 - Haftalık
3 - Aylık
Seçim (1/2/3): 2
🎯 Tarama Koşulunu Seçin:
1 - EMA1 EMA2'yi Yukarı Kesiyor
2 - Fiyat EMA1'i Yukarı Kesiyor
3 - Fiyat EMA1'e Yüzdesel Yakınlık (0-5%)
Seçiminiz (1/2/3): 1

📌 Kompozit Tarama Modunu Seçin:
1 - 🔁 Tüm Kompozit Hisseleri Tara
2 - 🔍 Belirli Kompozit Hisse(ler)i Tara (Bilgilendirme)
Seçiminiz (1/2): 1

📈 Seçilen EMA'lar → EMA1: 13, EMA2: 34, EMA3: 55 | Periyot: Haftalık
🔢 Toplam taranacak hisse sayısı: 601

🔍 Kompozit Tarama Başladı...


⏳ Taranıyor: 100%|██████████| 601/601 [10:27<00:00,  1.04s/it]


🎯 9 hisse bulundu.


,Hisse,Periyot,Fiyat,EMA34,Kriter,Teknik Sinyaller,RSI
0,BRKVY,Haftalık,65.90₺,0.61₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%),57.560000
1,EKIZ,Haftalık,63.90₺,0.61₺,"✅ EMA13, EMA34'yi yukarı kesti","✅ EMA13, EMA34'yi yukarı kesti | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%)",55.540000
2,CUSAN,Haftalık,23.20₺,0.23₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 💰 Yükseliş Destekli Hacim | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🔵 Momentum Pozitif (Alıcılı),52.570000
3,ARSAN,Haftalık,20.40₺,0.21₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🔵 Momentum Pozitif (Alıcılı),50.590000
4,CEMZY,Haftalık,15.89₺,0.15₺,"✅ EMA13, EMA34'yi yukarı kesti","✅ EMA13, EMA34'yi yukarı kesti | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%)",61.260000
5,BIGEN,Haftalık,12.99₺,0.13₺,"✅ EMA13, EMA34'yi yukarı kesti",🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📊 OBV Yükselişte | ⚡ Momentum Negatif | 🟠 Momentum Zayıf (Satıcılı),nan
6,CVKMD,Haftalık,11.98₺,0.11₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%),59.340000
7,AVGYO,Haftalık,8.99₺,0.09₺,"✅ EMA13, EMA34'yi yukarı kesti","✅ RSI > 50 🔼 | ✅ EMA13, EMA34'yi yukarı kesti | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%)",54.150000
8,BRMEN,Haftalık,5.75₺,0.06₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 🔄 MACD Sinyalin Üstünde (Ama 0 Altı) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%),54.960000



🏆 Fiyat Bazlı En Yüksek 10 Hisse


,Hisse,Periyot,Fiyat,EMA34,Kriter,Teknik Sinyaller,RSI
Sıra,,,,,,,
1,BRKVY,Haftalık,65.90₺,0.61₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%),57.560000
2,EKIZ,Haftalık,63.90₺,0.61₺,"✅ EMA13, EMA34'yi yukarı kesti","✅ EMA13, EMA34'yi yukarı kesti | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%)",55.540000
3,CUSAN,Haftalık,23.20₺,0.23₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 💰 Yükseliş Destekli Hacim | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🔵 Momentum Pozitif (Alıcılı),52.570000
4,ARSAN,Haftalık,20.40₺,0.21₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🔵 Momentum Pozitif (Alıcılı),50.590000
5,CEMZY,Haftalık,15.89₺,0.15₺,"✅ EMA13, EMA34'yi yukarı kesti","✅ EMA13, EMA34'yi yukarı kesti | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%)",61.260000
6,BIGEN,Haftalık,12.99₺,0.13₺,"✅ EMA13, EMA34'yi yukarı kesti",🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📊 OBV Yükselişte | ⚡ Momentum Negatif | 🟠 Momentum Zayıf (Satıcılı),nan
7,CVKMD,Haftalık,11.98₺,0.11₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%),59.340000
8,AVGYO,Haftalık,8.99₺,0.09₺,"✅ EMA13, EMA34'yi yukarı kesti","✅ RSI > 50 🔼 | ✅ EMA13, EMA34'yi yukarı kesti | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%)",54.150000
9,BRMEN,Haftalık,5.75₺,0.06₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 🔄 MACD Sinyalin Üstünde (Ama 0 Altı) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%),54.960000



✅ RSI 50 Üzeri Olan Hisseler


,Hisse,Periyot,Fiyat,EMA34,Kriter,Teknik Sinyaller,RSI
0,BRKVY,Haftalık,65.90₺,0.61₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%),57.560000
1,EKIZ,Haftalık,63.90₺,0.61₺,"✅ EMA13, EMA34'yi yukarı kesti","✅ EMA13, EMA34'yi yukarı kesti | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%)",55.540000
2,CUSAN,Haftalık,23.20₺,0.23₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 💰 Yükseliş Destekli Hacim | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🔵 Momentum Pozitif (Alıcılı),52.570000
3,ARSAN,Haftalık,20.40₺,0.21₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🔵 Momentum Pozitif (Alıcılı),50.590000
4,CEMZY,Haftalık,15.89₺,0.15₺,"✅ EMA13, EMA34'yi yukarı kesti","✅ EMA13, EMA34'yi yukarı kesti | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%)",61.260000
5,CVKMD,Haftalık,11.98₺,0.11₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%),59.340000
6,AVGYO,Haftalık,8.99₺,0.09₺,"✅ EMA13, EMA34'yi yukarı kesti","✅ RSI > 50 🔼 | ✅ EMA13, EMA34'yi yukarı kesti | 📈 Fiyat EMA55'ün üstünde | 📈 MACD Pozitif Bölgede (0 Üzeri) | 💪 MACD Momentum Yukarı | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%)",54.150000
7,BRMEN,Haftalık,5.75₺,0.06₺,"✅ EMA13, EMA34'yi yukarı kesti",✅ RSI > 50 🔼 | 🔻 EMA13 EMA34'nin altında | 📈 Fiyat EMA55'ün üstünde | 🔄 MACD Sinyalin Üstünde (Ama 0 Altı) | 💪 MACD Momentum Yukarı | 📊 OBV Yükselişte | 🚀 Momentum Pozitif | 🚀 Momentum Çok Güçlü (>+5%),54.960000
